# Third-order Ground-State Bleaching spectra of a monomer system

Importing useful classes, methods and modules:

In [1]:
from qudofedi import Qjob, FeynmanDiagram, System, PostProcess
from qutip import sigmaz
from qiskit import Aer
import numpy as np

System parameters:

In [2]:
e = 1.55                                     # Electronic energy gap [eV]
H = - e / 2 * sigmaz()                       # Hamiltonian of the system [eV]

mu = 1                                       # Electric dipole moment [a.u.]

system = System(H, mu)                       # System object

Spectroscopy parameters:

In [3]:
t_1_min = 0.                                 # Initial delay time (t1) [rad/eV]
t_1_max = 151.                               # Final delay time (t1) [rad/eV]
dt_1 = 15.                                   # Delta delay time (t1) [rad/eV]
t_1 = np.arange(t_1_min, t_1_max, dt_1)      # Delay time list (t1) [rad/eV]

t_2 = [0]                                    # Delay time list (t2) [rad/eV]

t_3_min = 0.                                 # Initial delay time (t3) [rad/eV]
t_3_max = 151.                               # Final delay time (t3) [rad/eV]
dt_3 = 15.                                   # Delta delay time (t3) [rad/eV]
t_3 = np.arange(t_3_min, t_3_max, dt_3)      # Delay time list (t3) [rad/eV]

t_list = [t_1, t_2, t_3]                     # List of delay times (t1, t2, t3) [rad/eV]

FD = FeynmanDiagram('gsb', t_list)           # FeynmanDiagram object

IBM Quantum backend:

In [4]:
backend = Aer.get_backend('qasm_simulator')

In [5]:
# from qiskit_ibm_provider import IBMProvider
# 
# provider = IBMProvider(instance='ibm-q/open/main')
# backend = provider.get_backend('ibmq_manila')

Create the Qjob object and getting the response function:

In [6]:
job = Qjob(system, FD)                       # Qjob object

response = job.run(backend,
                   shots = 4000,
                   save_Qjob = True,
                   save_name = 'example_gsb_TLS',
                   save_checkpoint = True,
                  )                          # Run the simulation returning a numpy.ndarray with the results

Total number of circuits = 242


Process ForkProcess-1:
Process ForkProcess-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/juu8242/opt/anaconda3/envs/qudofedi/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/juu8242/opt/anaconda3/envs/qudofedi/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/juu8242/opt/anaconda3/envs/qudofedi/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/juu8242/opt/anaconda3/envs/qudofedi/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/juu8242/opt/anaconda3/envs/qudofedi/lib/python3.10/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/juu8242/opt/anaconda3/envs/qudofedi/lib/python3.10/concurrent/futures/process.py", line 240, in _process_worker
    call_

Error during computation:  A process in the process pool was terminated abruptly while the future was running or pending.


Post-processing the results (rotating frame, exponential damping, zero-padding) to obtain the time oscillation and frequency spectra:

In [7]:
PostProcess(job,
            RF_freq = 1.505,
            damping_rate = 29.54 * 10**(-3),
            pad = 3,
           )                                 # Post-process and print the results

TypeError: 'NoneType' object is not subscriptable

## Adding readout-error mitigation

In [ ]:
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit_ibm_provider import IBMProvider

Select a real quantum processor:

In [ ]:
provider = IBMProvider(instance='ibm-q/open/main')
real_backend = provider.get_backend('ibmq_manila')

Create the noise model (not necessary if using the real_backend as the backend of computation):

In [ ]:
from qiskit_aer.noise.noise_model import NoiseModel

noise_model = NoiseModel().from_backend(real_backend)

Create the Qjob object and getting the response function:

In [ ]:
job = Qjob(system, FD)                       # Qjob object

response = job.run(backend,
                   shots = 4000,
                   save_Qjob = True,
                   save_name = 'example_gsb_TLS_mitigation',
                   save_checkpoint = True,
                   noise_model = noise_model,
                   coupling_map = real_backend.configuration().coupling_map,
                   initial_layout = [1, 2],    # An initial_layout specification is necessary to not get errors in Qiskit
                   measurement_error_mitigation_cls=CompleteMeasFitter,
                   measurement_error_mitigation_shots = 4000,
                  )                          # Run the simulation returning a numpy.ndarray with the results

In [ ]:
# For readout-error mitigation on a real device:
#
# job = Qjob(system, FD)                       # Qjob object
#
# response = job.run(backend,
#                    shots = 4000,
#                    save_Qjob = True,
#                    save_name = 'example_gsb_TLS_mitigation',
#                    save_checkpoint = True,
#                    initial_layout = [1, 2],    # An initial_layout specification is necessary to not get errors in Qiskit
#                    measurement_error_mitigation_cls=CompleteMeasFitter,
#                    measurement_error_mitigation_shots = 4000,
#                   )                          # Run the simulation returning a numpy.ndarray with the results

Post-processing the results (rotating frame, exponential damping, zero-padding) to obtain the time oscillation and frequency spectra:

In [ ]:
PostProcess(job,
            RF_freq = 1.505,
            damping_rate = 29.54 * 10**(-3),
            pad = 3,
           )                                 # Post-process and print the results